In [6]:
import cv2
import os

# Create directory for saving images
def create_dataset_dir(name):
    if not os.path.exists(f"dataset/{name}"):
        os.makedirs(f"dataset/{name}")

def capture_images(name):
    cap = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    create_dataset_dir(name)
    img_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            img_id += 1
            face = gray[y:y+h, x:x+w]
            file_name_path = f"dataset/{name}/{img_id}.jpg"
            cv2.imwrite(file_name_path, face)

            cv2.putText(frame, str(img_id), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        cv2.imshow("Face", frame)

        if cv2.waitKey(1) == 27 or img_id == 500:  # ESC to break or 100 images captured
            break

    cap.release()
    cv2.destroyAllWindows()

# Capture images
name = input("Enter the name of the person: ")
capture_images(name)


No faces found in the dataset. Check the dataset path and images.


In [8]:
import cv2
import numpy as np
import os

def get_images_and_labels(dataset_path):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    image_paths = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) if f.endswith('.jpg')]
    face_samples = []
    ids = []
    
    for image_path in image_paths:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        img_np = np.array(img, 'uint8')
        id = int(os.path.split(image_path)[-1].split(".")[0])
        faces = face_cascade.detectMultiScale(img_np, scaleFactor=1.3, minNeighbors=5)
        for (x, y, w, h) in faces:
            face_samples.append(img_np[y:y+h, x:x+w])
            ids.append(id)
    
    return face_samples, ids

def train_model():
    dataset_path = 'dataset'
    
    # Ensure 'trainer' directory exists
    if not os.path.exists('trainer'):
        os.makedirs('trainer')
        
    recognizer = cv2.face.LBPHFaceRecognizer_create()

    for person_name in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person_name)
        if os.path.isdir(person_path):  # Ensure it's a directory
            print(f"Training for {person_name}...")
            faces, ids = get_images_and_labels(person_path)
            if len(faces) > 0 and len(ids) > 0:
                recognizer.train(faces, np.array(ids))
                recognizer.save(f'trainer/{person_name}_model.yml')
                print(f"Model for {person_name} saved.")
            else:
                print(f"No valid data found for {person_name}")
        else:
            print(f"{person_name} is not a valid directory")

    print("Model training completed.")

# Train the model
train_model()


Training for Siddhartha...
Model for Siddhartha saved.
Model training completed.


In [9]:
import cv2
import os

def recognize_faces():
    # Create an LBPH face recognizer instance
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    # Load Haar Cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Dictionary to map face ID to name
    id_to_name = {}
    
    # Load all trained models and their names
    for file in os.listdir('trainer'):
        if file.endswith('_model.yml'):
            name = file.split('_')[0]
            recognizer.read(f'trainer/{file}')
            id_to_name[len(id_to_name)] = name  # Use sequential IDs for names

    print("Loaded names:", id_to_name)  # Debugging line

    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            face_region = gray[y:y+h, x:x+w]
            face_region_resized = cv2.resize(face_region, (200, 200))  # Resize to a consistent size
            face_id, confidence = recognizer.predict(face_region_resized)
            
            if confidence < 100:
                name = id_to_name.get(face_id, "Unknown")
            else:
                name = "Unknown"
            
            cv2.putText(frame, name, (x+5, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"{round(100 - confidence)}%", (x+5, y+h-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        cv2.imshow("Face Recognition", frame)

        if cv2.waitKey(1) == 27:  # ESC to break
            break

    cap.release()
    cv2.destroyAllWindows()

# Start face recognition
recognize_faces()


Loaded names: {0: 'Siddhartha'}


In [10]:
import cv2
import os

def capture_images(person_name):
    # Create a directory for the person if it doesn't exist
    dataset_path = 'dataset'
    person_path = os.path.join(dataset_path, person_name)
    if not os.path.exists(person_path):
        os.makedirs(person_path)

    cap = cv2.VideoCapture(0)
    count = 1

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        cv2.imshow('Capture Image', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
            break
        
        image_path = os.path.join(person_path, f'{person_name}_{count}.jpg')
        cv2.imwrite(image_path, frame)
        count += 1
    
    cap.release()
    cv2.destroyAllWindows()

# Capture images for a person named Siddhartha
capture_images('Siddhartha')


In [14]:
import cv2
from deepface import DeepFace
import os

def recognize_faces():
    # Directory where face images are stored
    face_database_path = 'face_database'
    
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Save the current frame as a temporary file
        temp_frame_path = 'temp_frame.jpg'
        cv2.imwrite(temp_frame_path, rgb_frame)

        try:
            # Perform face recognition with a specified model
            result = DeepFace.find(temp_frame_path, db_path=face_database_path, model_name='Facenet')

            if len(result) > 0:
                name = result[0]['identity']
            else:
                name = "Unknown"
        except Exception as e:
            print(f"Error in DeepFace: {e}")
            name = "Unknown"

        # Display the recognized name on the frame
        cv2.putText(frame, name, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('Face Recognition', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    
    # Remove temporary frame
    os.remove(temp_frame_path)

# Start face recognition
recognize_faces()


24-09-19 21:09:09 - Found 726 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations:   0%|          | 0/726 [00:00<?, ?it/s]


Error in DeepFace: module 'deepface.modules.modeling' has no attribute 'build_model'
24-09-19 21:09:09 - Found 726 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations:   0%|          | 0/726 [00:00<?, ?it/s]

Error in DeepFace: module 'deepface.modules.modeling' has no attribute 'build_model'


...

Error in DeepFace: module 'deepface.modules.modeling' has no attribute 'build_model'
24-09-19 21:09:29 - Found 726 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations:   0%|          | 0/726 [00:00<?, ?it/s]


Error in DeepFace: module 'deepface.modules.modeling' has no attribute 'build_model'
24-09-19 21:09:29 - Found 726 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations:   0%|          | 0/726 [00:00<?, ?it/s]


Error in DeepFace: module 'deepface.modules.modeling' has no attribute 'build_model'


In [37]:
import cv2
import os

# Create a folder for the user's dataset
def create_dataset_dir(name):
    path = f"dataset/{name}"
    if not os.path.exists(path):
        os.makedirs(path)
    return path

# Capture face images for the given name and save them in the dataset folder
def capture_images(name):
    cap = cv2.VideoCapture(0)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    dataset_path = create_dataset_dir(name)
    img_id = 0

    print(f"Capturing images for {name}. Press ESC to stop.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            img_id += 1
            face = gray[y:y+h, x:x+w]
            file_name_path = os.path.join(dataset_path, f"{img_id}.jpg")
            cv2.imwrite(file_name_path, face)

            # Display the face and ID in the live camera feed
            cv2.putText(frame, str(img_id), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        cv2.imshow("Face Capture", frame)

        if cv2.waitKey(1) == 27 or img_id >= 100:  # ESC to stop or after 100 images
            break

    cap.release()
    cv2.destroyAllWindows()

# Main function to capture images
name = input("Enter the name of the person: ")
capture_images(name)


Capturing images for . Press ESC to stop.


In [35]:
import cv2
import numpy as np
import os

# Function to get face images and labels for training
def get_images_and_labels(dataset_path):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    face_samples = []
    ids = []

    # Loop over each person's folder in the dataset
    for person_name in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person_name)
        if os.path.isdir(person_path):
            for image_file in os.listdir(person_path):
                if image_file.endswith('.jpg'):
                    image_path = os.path.join(person_path, image_file)
                    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                    faces = face_cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=5)

                    for (x, y, w, h) in faces:
                        face_samples.append(img[y:y+h, x:x+w])
                        ids.append(int(os.path.split(image_file)[-1].split('.')[0]))

    return face_samples, ids

# Train the face recognition model on captured images
def train_model():
    dataset_path = 'dataset'
    if not os.path.exists('trainer'):
        os.makedirs('trainer')

    recognizer = cv2.face.LBPHFaceRecognizer_create()
    
    faces, ids = get_images_and_labels(dataset_path)
    
    if len(faces) > 0 and len(ids) > 0:
        recognizer.train(faces, np.array(ids))
        recognizer.save('trainer/face_model.yml')
        print("Model training completed.")
    else:
        print("No valid images to train on.")

# Main function to train the model
train_model()


Training for Siddhartha...
Processing images in: dataset\Siddhartha
No faces detected in image: dataset\Siddhartha\1.jpg
No faces detected in image: dataset\Siddhartha\10.jpg
No faces detected in image: dataset\Siddhartha\100.jpg
No faces detected in image: dataset\Siddhartha\11.jpg
No faces detected in image: dataset\Siddhartha\12.jpg
No faces detected in image: dataset\Siddhartha\13.jpg
No faces detected in image: dataset\Siddhartha\14.jpg
No faces detected in image: dataset\Siddhartha\15.jpg
No faces detected in image: dataset\Siddhartha\16.jpg
No faces detected in image: dataset\Siddhartha\17.jpg
No faces detected in image: dataset\Siddhartha\18.jpg
No faces detected in image: dataset\Siddhartha\19.jpg
No faces detected in image: dataset\Siddhartha\20.jpg
No faces detected in image: dataset\Siddhartha\21.jpg
No faces detected in image: dataset\Siddhartha\26.jpg
No faces detected in image: dataset\Siddhartha\27.jpg
No faces detected in image: dataset\Siddhartha\30.jpg
No faces detec

In [36]:
import cv2
import os

# Recognize faces using the trained LBPH model
def recognize_faces():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('trainer/face_model.yml')

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in faces:
            face_region = gray[y:y+h, x:x+w]
            face_id, confidence = recognizer.predict(face_region)
            
            if confidence < 100:  # Adjust confidence threshold as needed
                cv2.putText(frame, f"Matched (Confidence: {round(100 - confidence)}%)", (x+5, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "Not Matched", (x+5, y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        cv2.imshow("Face Recognition", frame)

        if cv2.waitKey(1) & 0xFF == 27:  # ESC to break
            break

    cap.release()
    cv2.destroyAllWindows()

# Start recognizing faces
recognize_faces()


Loaded names: {0: 'Siddhartha'}
22
Predicted ID: 22, Confidence: 46.967718266565235
2
Predicted ID: 2, Confidence: 45.56411594710832
2
Predicted ID: 2, Confidence: 45.56411594710832
2
Predicted ID: 2, Confidence: 44.46232325192829
2
Predicted ID: 2, Confidence: 44.46232325192829
2
Predicted ID: 2, Confidence: 44.714162721502106
2
Predicted ID: 2, Confidence: 44.714162721502106
2
Predicted ID: 2, Confidence: 45.175377951687715
2
Predicted ID: 2, Confidence: 45.175377951687715
2
Predicted ID: 2, Confidence: 42.5737482296393
2
Predicted ID: 2, Confidence: 42.5737482296393
22
Predicted ID: 22, Confidence: 44.84917020409013
22
Predicted ID: 22, Confidence: 44.84917020409013
2
Predicted ID: 2, Confidence: 45.19101316686221
2
Predicted ID: 2, Confidence: 45.19101316686221
2
Predicted ID: 2, Confidence: 46.10326784841669
2
Predicted ID: 2, Confidence: 46.10326784841669
24
Predicted ID: 24, Confidence: 47.08723994272097
24
Predicted ID: 24, Confidence: 47.08723994272097
22
Predicted ID: 22, Con